In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from sympy.abc import lamda

In [70]:
province = gpd.read_file(
    r"C:\Users\MUICT\Desktop\STONES\NASA_data\tha_admbnda_adm1_rtsd_20190221.shp")
province = province.to_crs("EPSG:4326")
# 1. Load the Thailand shapefile

In [4]:
 # national boundary


for var in ['vap',]:
    temp_df = pd.DataFrame()
    for year in range(2002, 2024):
        try:
            ds = xr.open_dataset(f"pre_data/TerraClimate_{var}_{year}.nc")
        except:
            print(f"{var} No data available for year {year}")
            continue

        raster = ds[var]
        raster.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        raster.rio.write_crs("EPSG:4326", inplace=True)
        raster_thailand = raster.rio.clip(thailand.geometry, thailand.crs)
        df = raster_thailand.to_dataframe(name=var).reset_index()
        df = df.loc[df[var].notnull()].reset_index(drop=True)
        df_gdp = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df["lon"], df["lat"]),
            crs="EPSG:4326"
        )

        # spatial join: assign each pixel to a province
        df_joined = gpd.sjoin(
            df_gdp,
            province[["ADM1_EN", "geometry"]],
            # how="inner",
            # predicate="within"
        )

        # province mean
        province_mean = (
            df_joined
            .groupby(["ADM1_EN",'time'])[var]
            .mean()
            .reset_index()
            .rename(columns={"ADM1_EN": "province", var: f"{var}"})
        )
        province_mean['month'] = province_mean['time'].dt.month
        province_mean['year'] = province_mean['time'].dt.year
        province_mean = province_mean.drop(columns=['time'])
        temp_df = pd.concat([temp_df, province_mean], axis=0)
    temp_df.to_csv(f'data/terraclimate_{var}_province_mean.csv', index=False)



vap No data available for year 2002
vap No data available for year 2011


In [ ]:

for var in ['tmax','tmin','ppt',]:
    temp_df = pd.DataFrame()
    for year in [2024]:
        try:
            ds = xr.open_dataset(f"pre_data/TerraClimate_{var}_{year}.nc")
        except:
            print(f"{var} No data available for year {year}")
            continue

        raster = ds[var]
        raster.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        raster.rio.write_crs("EPSG:4326", inplace=True)
        raster_thailand = raster.rio.clip(thailand.geometry, thailand.crs)
        df = raster_thailand.to_dataframe(name=var).reset_index()
        df = df.loc[df[var].notnull()].reset_index(drop=True)
        df_gdp = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df["lon"], df["lat"]),
            crs="EPSG:4326"
        )

        # spatial join: assign each pixel to a province
        df_joined = gpd.sjoin(
            df_gdp,
            province[["ADM1_EN", "geometry"]],
            # how="inner",
            # predicate="within"
        )

        # province mean
        province_mean = (
            df_joined
            .groupby(["ADM1_EN",'time'])[var]
            .mean()
            .reset_index()
            .rename(columns={"ADM1_EN": "province", var: f"{var}"})
        )
        province_mean['month'] = province_mean['time'].dt.month
        province_mean['year'] = province_mean['time'].dt.year
        province_mean = province_mean.drop(columns=['time'])
        temp_df = pd.concat([temp_df, province_mean], axis=0)
    temp_df.to_csv(f'data/terraclimate_{var}_province_mean_2024.csv', index=False)



In [17]:
final_df = pd.read_csv(f'data/terraclimate_vap_province_mean.csv')
for var in ['tmax','tmin','ppt','soil','ws']:
    df_var = pd.read_csv(f'data/terraclimate_{var}_province_mean.csv')
    final_df = pd.merge(final_df, df_var, on=['province','month','year'], how='left')
final_df = final_df.rename(columns={
    'tmax':'TMAX',
    'tmin':'TMIN',
    'ppt':'PPT',
    'soil':'SOIL',
    'vap':'VAP',
    'ws':'WS',
})


In [20]:
for var in ['tmax','tmin','ppt','soil','ws','vap','vpd']:
    df = pd.read_csv(f'data/terraclimate_{var}_province_mean.csv')
    df = df[['year','month','province',var]]
    df.to_csv(f'data/terraclimate_{var}_province_mean.csv', index=False)

In [34]:

for var in ['tmax','tmin','ppt','soil','vap','vpd','ws']:
    df = pd.read_csv(f'data/Final_data_province_mean_{var}.csv')
    df = df[df['year']>=2003]
    df.to_csv(f'data/Final_data_province_mean_{var}_2003onward.csv', index=False)
merged_df = pd.read_csv('data/Final_data_province_mean_ppt_2003onward.csv')
merged_df = merged_df.replace('Loburi','Lop Buri')
for var in ['tmax','tmin','soil','vap','vpd','ws']:
    df_var = pd.read_csv(f'data/Final_data_province_mean_{var}_2003onward.csv')
    df_var = df_var.replace('Loburi','Lop Buri')
    merged_df = pd.merge(merged_df, df_var, on=['province','month','year'], how='left')
merged_df['TAVG'] = (merged_df['tmax'] + merged_df['tmin']) /2
merged_df = merged_df.rename(columns={
    'tmax':'TMAX',
    'tmin':'TMIN',
    'ppt':'PPT',
    'soil':'SOIL',
    'vap':'VAP',
    'ws':'WS',
    'vpd':'VPD',
})

In [67]:

merged_df= merged_df.sort_values(by=['province','year','month']).reset_index(drop=True)

for province in merged_df['province'].unique():
    province_df = merged_df[merged_df['province']==province]
    for i in province_df['year'].unique():
        print(province, i)


    break

Amnat Charoen 2003
Amnat Charoen 2004
Amnat Charoen 2005
Amnat Charoen 2006
Amnat Charoen 2007
Amnat Charoen 2008
Amnat Charoen 2009
Amnat Charoen 2010
Amnat Charoen 2011
Amnat Charoen 2012
Amnat Charoen 2013
Amnat Charoen 2014
Amnat Charoen 2015
Amnat Charoen 2016
Amnat Charoen 2017
Amnat Charoen 2018
Amnat Charoen 2019
Amnat Charoen 2020
Amnat Charoen 2021
Amnat Charoen 2022
Amnat Charoen 2023


In [64]:
merged_df[['province','year','month','PPT','ppt_rolling_sum']]

,province,year,month,PPT,ppt_rolling_sum
0,Amnat Charoen,2003,1,0.117361,NaN
1,Amnat Charoen,2003,2,30.798264,NaN
2,Amnat Charoen,2003,3,50.106250,81.021875
3,Amnat Charoen,2003,4,63.870139,144.774653
4,Amnat Charoen,2003,5,219.253125,333.229514
...,...,...,...,...,...
19399,Yasothon,2023,8,220.860714,698.935204
19400,Yasothon,2023,9,386.003061,865.257653
19401,Yasothon,2023,10,125.733673,732.597449
19402,Yasothon,2023,11,7.355612,519.092347
